In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.path import Path
from matplotlib.patches import PathPatch

from shapely.geometry import Polygon

import geopandas as gpd

import folium

pd.set_option('display.max_columns', None) ### This line makes all the columns display, rather than ellipses shorten

In [2]:
from pyproj import Transformer

# WGS84 (lat/lon) <-> Web Mercator (meters)
to_3857 = Transformer.from_crs("EPSG:4326", "EPSG:3857", always_xy=True)
to_4326 = Transformer.from_crs("EPSG:3857", "EPSG:4326", always_xy=True)

def project_to_3857(lon, lat):
    return to_3857.transform(lon, lat)

def project_to_4326(x, y):
    return to_4326.transform(x, y)

In [3]:
def bezier_curve(start, control1, control2, end, n=100):
    """Generate a cubic Bezier curve."""
    t = np.linspace(0, 1, n)[:, None]  # shape (n, 1)
    start, control1, control2, end = map(np.array, [start, control1, control2, end])
    curve = (
        (1 - t)**3 * start
        + 3 * (1 - t)**2 * t * control1
        + 3 * (1 - t) * t**2 * control2
        + t**3 * end
    )
    return curve


In [4]:
import numpy as np
from shapely.geometry import Polygon

def perp(v):
    return np.array([-v[1], v[0]])  # rotate 90° left

def flow_patch_geo(
    start_ll, end_ll,
    start_width, end_width,
    bend=0.2,
    curvature=150,
    inherit_normal=None,   # optional: shared normal for siblings
):
    # ---- Project lon/lat to meters ----
    start = np.array(project_to_3857(*start_ll))
    end   = np.array(project_to_3857(*end_ll))

    # Direction vector
    d = end - start

    # ---- Inline 'unit' function ----
    d_norm = np.linalg.norm(d)
    if d_norm == 0:
        raise ValueError("Start and end coordinates are identical; cannot compute direction.")
    ud = d / d_norm  # unit direction vector

    # Perpendicular (normal) direction
    if inherit_normal is None:
        n = perp(ud)
    else:
        # Use parent's/trunk's normal, but normalize it
        n = inherit_normal / np.linalg.norm(inherit_normal)

    # Control points
    ctrl1 = start + d * bend + n * curvature
    ctrl2 = end   - d * bend + n * curvature

    # Build Bezier edges
    top = bezier_curve(
        start + n * (start_width/2),
        ctrl1 + n * (start_width/2),
        ctrl2 + n * (end_width/2),
        end   + n * (end_width/2)
    )

    bottom = bezier_curve(
        end   - n * (end_width/2),
        ctrl2 - n * (end_width/2),
        ctrl1 - n * (start_width/2),
        start - n * (start_width/2)
    )

    # Combine vertices
    verts = np.vstack([top, bottom, top[0]])

    # Convert back to lon/lat
    verts_ll = [project_to_4326(x, y) for x, y in verts]

    # Return polygon + this flow's normal for children
    return Polygon(verts_ll), n

In [5]:
'''
def flow_patch_geo(
    start_ll, end_ll,
    start_width, end_width,
    bend=0.2,
    curvature=150,
    inherit_normal=None,    # NEW: pass a shared normal vector
):
    # Project to meters
    start = np.array(project_to_3857(*start_ll))
    end   = np.array(project_to_3857(*end_ll))

    # Branch direction
    d = end - start
    ud = unit(d)

    # If no inherited normal is supplied, use this segment's perpendicular
    if inherit_normal is None:
        n = perp(ud)
    else:
        # Use the trunk's exact orientation
        n = inherit_normal / np.linalg.norm(inherit_normal)

    # Control points
    ctrl1 = start + d * bend + n * curvature
    ctrl2 = end   - d * bend + n * curvature

    # Build Bezier edges using the shared normal n
    top = bezier_curve(
        start + n * (start_width/2),
        ctrl1 + n * (start_width/2),
        ctrl2 + n * (end_width/2),
        end   + n * (end_width/2)
    )

    bottom = bezier_curve(
        end   - n * (end_width/2),
        ctrl2 - n * (end_width/2),
        ctrl1 - n * (start_width/2),
        start - n * (start_width/2)
    )

    verts = np.vstack([top, bottom, top[0]])
    verts_ll = [project_to_4326(x, y) for x, y in verts]

    return Polygon(verts_ll), n  # return polygon and this flow’s normal
    '''

"\ndef flow_patch_geo(\n    start_ll, end_ll,\n    start_width, end_width,\n    bend=0.2,\n    curvature=150,\n    inherit_normal=None,    # NEW: pass a shared normal vector\n):\n    # Project to meters\n    start = np.array(project_to_3857(*start_ll))\n    end   = np.array(project_to_3857(*end_ll))\n\n    # Branch direction\n    d = end - start\n    ud = unit(d)\n\n    # If no inherited normal is supplied, use this segment's perpendicular\n    if inherit_normal is None:\n        n = perp(ud)\n    else:\n        # Use the trunk's exact orientation\n        n = inherit_normal / np.linalg.norm(inherit_normal)\n\n    # Control points\n    ctrl1 = start + d * bend + n * curvature\n    ctrl2 = end   - d * bend + n * curvature\n\n    # Build Bezier edges using the shared normal n\n    top = bezier_curve(\n        start + n * (start_width/2),\n        ctrl1 + n * (start_width/2),\n        ctrl2 + n * (end_width/2),\n        end   + n * (end_width/2)\n    )\n\n    bottom = bezier_curve(\n     

In [6]:
'''
import numpy as np
from shapely.geometry import Polygon

def unit(v):
    return v / np.linalg.norm(v)

def perp(v):
    return np.array([-v[1], v[0]])  # rotate 90° left

def flow_patch_geo(
    start_ll, end_ll,
    start_width, end_width,
    bend=0.2,             # 0 = straight line, 0.3 = nice curve
    curvature=150,      # meters of sideways bend
):

    # ---- Project lon/lat to meters ----
    start = np.array(project_to_3857(*start_ll))
    end   = np.array(project_to_3857(*end_ll))

    # Direction vector
    d = end - start
    ud = unit(d)

    # Perpendicular
    n = perp(ud)

    # Control point positions:
    ctrl1 = start + d * bend + n * curvature
    ctrl2 = end   - d * bend + n * curvature

    # Build Bezier edges
    top = bezier_curve(
        start + n * (start_width/2),
        ctrl1 + n * (start_width/2),
        ctrl2 + n * (end_width/2),
        end   + n * (end_width/2)
    )

    bottom = bezier_curve(
        end   - n * (end_width/2),
        ctrl2 - n * (end_width/2),
        ctrl1 - n * (start_width/2),
        start - n * (start_width/2)
    )

    # Combine vertices
    verts = np.vstack([top, bottom, top[0]])

    # Back to lon/lat
    verts_ll = [project_to_4326(x, y) for x, y in verts]

    poly = Polygon(verts_ll)
    return poly
    '''

'\nimport numpy as np\nfrom shapely.geometry import Polygon\n\ndef unit(v):\n    return v / np.linalg.norm(v)\n\ndef perp(v):\n    return np.array([-v[1], v[0]])  # rotate 90° left\n\ndef flow_patch_geo(\n    start_ll, end_ll,\n    start_width, end_width,\n    bend=0.2,             # 0 = straight line, 0.3 = nice curve\n    curvature=150,      # meters of sideways bend\n):\n\n    # ---- Project lon/lat to meters ----\n    start = np.array(project_to_3857(*start_ll))\n    end   = np.array(project_to_3857(*end_ll))\n\n    # Direction vector\n    d = end - start\n    ud = unit(d)\n\n    # Perpendicular\n    n = perp(ud)\n\n    # Control point positions:\n    ctrl1 = start + d * bend + n * curvature\n    ctrl2 = end   - d * bend + n * curvature\n\n    # Build Bezier edges\n    top = bezier_curve(\n        start + n * (start_width/2),\n        ctrl1 + n * (start_width/2),\n        ctrl2 + n * (end_width/2),\n        end   + n * (end_width/2)\n    )\n\n    bottom = bezier_curve(\n        end

In [7]:
## make a list of points 


## E line going from south to north
point_list = [
     (47.683729430644014, -122.34433918947462),
     (47.687259045590366, -122.34435249866571),
     (47.69108874059965, -122.34437859849116),
]

In [8]:
point_list[0][1]

-122.34433918947462

In [9]:
Starting_Y_Val = point_list[0][0]
Second_Y_Val = point_list[1][0]
Third_Y_Val = point_list[2][0]

Starting_X_Val = point_list[0][1]
Second_X_Val = point_list[1][1]
Third_X_Val = point_list[2][1]

In [10]:

offset_const = 0.0005

trunk_poly, trunk_normal = flow_patch_geo(
#Z_line_polygons.append(flow_patch_geo(
    #start_ll=(-120, 40),
    #end_ll=(-118.5, 41),
    start_ll = (Starting_X_Val, Starting_Y_Val),
    end_ll = (Second_X_Val, Second_Y_Val),
    start_width=80,
    end_width=80,
    bend=0.2,
    curvature=20,
)

branch_poly, branch_normal = flow_patch_geo(
#Z_line_polygons.append(flow_patch_geo(
    #start_ll=(-118.5, 41),
    #end_ll=(-117.7, 41.6),
    start_ll = (Starting_X_Val - offset_const * 5, Starting_Y_Val),
    end_ll = (Second_X_Val - offset_const, Second_Y_Val),
    start_width=40,
    end_width=40,
    bend=0.7,
    curvature=50,
    inherit_normal=trunk_normal,   # <<--- KEY FIX
)

In [11]:
Starting_Y_Val

47.683729430644014

In [12]:
'''
E_line_polygons = []

#Starting_Y_Val = 47.683729430644014
#Second_Y_Val = 47.687259045590366
#Third_Y_Val = 47.69108874059965

#Starting_Y_Val = point_list[0][1]
#Second_Y_Val = point_list[1][1]
#Third_Y_Val = point_list[2][1]


offset_const = 0.0005


### going to first stop
##main line
E_line_polygons.append(flow_patch_geo(
    start_ll = (Starting_X_Val, Starting_Y_Val),
    end_ll = (Second_X_Val, Second_Y_Val),
    start_width = 150,
    end_width = 150,
    bend = 0,
    curvature = 0
))

## boarding line
E_line_polygons.append(flow_patch_geo(
    start_ll = (Second_X_Val - offset_const * 5, Starting_Y_Val),
    end_ll = (Third_X_Val - offset_const, Second_Y_Val),
    start_width = 20,
    end_width = 20,
    bend = 0.4,
    curvature = -30
))

### going to second stop

#main line
E_line_polygons.append(flow_patch_geo(
    start_ll = (-122.34435249866571, Second_Y_Val),
    end_ll = (-122.34437859849116, Third_Y_Val),
    start_width = 110,
    end_width = 110,
    bend = 0,
    curvature = 0
))
'''

'\nE_line_polygons = []\n\n#Starting_Y_Val = 47.683729430644014\n#Second_Y_Val = 47.687259045590366\n#Third_Y_Val = 47.69108874059965\n\n#Starting_Y_Val = point_list[0][1]\n#Second_Y_Val = point_list[1][1]\n#Third_Y_Val = point_list[2][1]\n\n\noffset_const = 0.0005\n\n\n### going to first stop\n##main line\nE_line_polygons.append(flow_patch_geo(\n    start_ll = (Starting_X_Val, Starting_Y_Val),\n    end_ll = (Second_X_Val, Second_Y_Val),\n    start_width = 150,\n    end_width = 150,\n    bend = 0,\n    curvature = 0\n))\n\n## boarding line\nE_line_polygons.append(flow_patch_geo(\n    start_ll = (Second_X_Val - offset_const * 5, Starting_Y_Val),\n    end_ll = (Third_X_Val - offset_const, Second_Y_Val),\n    start_width = 20,\n    end_width = 20,\n    bend = 0.4,\n    curvature = -30\n))\n\n### going to second stop\n\n#main line\nE_line_polygons.append(flow_patch_geo(\n    start_ll = (-122.34435249866571, Second_Y_Val),\n    end_ll = (-122.34437859849116, Third_Y_Val),\n    start_width =

In [13]:
colors = ['cornflowerblue', 'orange', 'cornflowerblue']

In [20]:
m = folium.Map(location=[47.65, -122.4], zoom_start=12)

#for poly in geoms:
folium.GeoJson(trunk_poly).add_to(m)

folium.GeoJson(branch_poly).add_to(m)


m

In [15]:
'''
m = folium.Map(location=[47.683729430644014, -122.34433918947462], zoom_start=13)

for poly, color in zip(E_line_polygons, colors):
    folium.GeoJson(
        poly,
        style_function=lambda _, col = color: {
            "fillColor": col,
            "color": col,
            "weight": 1,
            "fillOpacity": 0.7
        }
    ).add_to(m)

m'''

'\nm = folium.Map(location=[47.683729430644014, -122.34433918947462], zoom_start=13)\n\nfor poly, color in zip(E_line_polygons, colors):\n    folium.GeoJson(\n        poly,\n        style_function=lambda _, col = color: {\n            "fillColor": col,\n            "color": col,\n            "weight": 1,\n            "fillOpacity": 0.7\n        }\n    ).add_to(m)\n\nm'